# GlusterFS安装报告（基于Kubernetes）
## 什么是GlusterFS
>**GlusterFS (Gluster File System)** 是一个开源的分布式文件系统，主要由 Z RESEARCH 公司负责开发。GlusterFS 是 Scale-Out 存储解决方案 Gluster 的核心，具有强大的横向扩展能力，通过扩展能够支持数PB存储容量和处理数千客户端。GlusterFS 借助 TCP/IP 或 InfiniBand RDMA 网络将物理分布的存储资源聚集在一起，使用单一全局命名空间来管理数据。GlusterFS 基于可堆叠的用户空间设计，可为各种不同的数据负载提供优异的性能。

<p>GlusterFS由存储服务器（Brick Server），客户端以及NFS/Samba存储网关组成。GlusterFS最大的设计特点就是其没有元数据（metadata）存储组件。这样的设计对整个系统性能，可靠性和稳定性的提升都具有非常重大的意义。其总体架构如下图所示：</p>

<p>接下来简要说明一下Glusterfs中的一些要用到的专业术语：<ul><li><b>Brick</b>：最基本的存储单元，供客户端挂载使用。</li></ul><ul><li><b>Fuse</b>：类Unix操作系统中的一个可动态加载的模块，用以允许用户不用修改内核即可创建自己的文件系统。</li></ul><ul><li><b>PV(phyical volume)</b>：即物理卷，是<b>lvm（逻辑卷管理系统）</b>中的概念，类似于物理磁盘上的分区，其基本组成单位是物理快（PE），其默认大小为4MB。</li><li><b>VG(volumes group)</b>：即<b>卷组</b>，由物理卷构成，是逻辑卷的组成部分。</li></ul><ul><li><b>LV(Logistic Volume)逻辑卷</b>：是lvm中最上层的文件卷，由若干个卷组中的物理卷构成，其大小可以动态指定，也能够扩大或缩小。</li></ul>Glusterfs一共支持七种挂载卷的形式，包括分布式卷，条带卷，副本卷等多种类型，具体的内容可参见<a href="https://www.ibm.com/developerworks/cn/opensource/os-cn-glusterfs-docker-volume/index.html?lnk=hm" target="_blank">基于 GlusterFS 实现 Docker 集群的分布式存储</a>这篇文章。</p>

## 基于Kubernetes的GlusterFS的安装
### 安装环境
<p><pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
NAME   STATUS    ROLES    VERSION   EXTERNAL-IP    OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
lab1   Ready     node     v1.8.2    172.19.0.131   Ubuntu 16.04.4 LTS   4.4.0-62-generic   docker://17.11.0-ce
lab2   Ready     node     v1.8.2    172.19.0.132   Ubuntu 16.04.2 LTS   4.4.0-62-generic   docker://17.11.0-ce
lab3   Ready     node     v1.8.2    172.19.0.133   Ubuntu 16.04.2 LTS   4.4.0-62-generic   docker://17.11.0-ce
lab4   Ready     master   v1.8.2    172.19.0.134   Ubuntu 16.04.4 LTS   4.4.0-62-generic   docker://17.11.0-ce
</pre>可以看到，安装环境中一共有四个节点，其中lab4为主节点，其余三个节点为从节点。每个节点上都运行Ubuntu16.04的操作系统。<b>（注意：GlusterFS安装要求必须至少有三个存储节点）</b></p>

### GlusterFS集群搭建
<p>如果计算机中没有预先编写好的yaml文件，可以从git上下载官方的配置仓库，在cmd中运行<code style="font-family:Consolas;padding:5px 5px 5px 5px;background-color:#F4F4F4">git clone https://github.com/gluster/gluster-kubernetes</code>即可下载到本地，其具体的yaml文件在配置仓库的<code style="font-family:Consolas;padding:5px 5px 5px 5px;background-color:#F4F4F4">./deploy/kube-templates</code>中。虽然GlusterFS提供安装指南，但还是推荐使用预先编写好的yaml文件配置GlusterFS，因为官方仓库里下载的配置文件还需要一定的更改才能够正确运行在本地集群上。下面的安装全都假定操作者已经拥有预先编写好的yaml文件或者已经对git上下载的配置文件进行了正确修改。</p>
<p>首先查看所有四个节点的hosts文件，需要在各个节点的hosts文件中添加其余节点的主机名，ip地址映射列表。例如在笔者所在的集群环境下hosts文件的内容：